In [2]:
using PyPlot

In [3]:
# Convert an Float32 rbg image to Float64 grayscale image
function convert_to_grayscale(I::Array{Float32,3})
    I=convert(Array{Float64,3}, I)
    I_gray = 0.2989*I[:,:,1] + 0.5870*I[:,:,2] + 0.1140*I[:,:,3]
    return I_gray::Array{Float64,2}
end

convert_to_grayscale (generic function with 1 method)

In [4]:
# Load Tsukuba disparity dataset and convert it to grayscale
function load_data()
    i0_path = string(@__DIR__,"/i0.png")
    i0 = imread(i0_path)
    i0 = convert_to_grayscale(i0)
    i1_path = string(@__DIR__,"/i1.png")
    i1 = imread(i1_path)
    i1 = convert_to_grayscale(i1)
    gt_path = string(@__DIR__,"/gt.png")
    gt64 = convert(Array{Float64,2}, imread(gt_path)*255)

    @assert maximum(gt64) <= 16
    return i0::Array{Float64,2}, i1::Array{Float64,2}, gt64::Array{Float64,2}
end

load_data (generic function with 1 method)

In [5]:
function log_studentt(x::Array{Float64,2}, alpha::Float64, sigma::Float64)

    # ??
    
    function log_student(x, alpha, sigma)
        return log((1 + (1/2*sigma^2)*x^2)^(-1*alpha))
    end
    
    function log_grad_student(x, alpha, sigma)
        return (-1*alpha*(1/sigma^2)*x) * (1 + (1/2*sigma^2)*x^2)^(-1*alpha - 2)
    end
    
    value = Array{Float64,2}(size(x))
    grad = Array{Float64,2}(size(x))
    for a = 1:size(x)[1]
        for b = 1:size(x)[b]
            value[a,b]= student_t(x[a,b], alpha, sigma))
            grad[a, b]= grad_student_t(x[a, b], alpha, sigma)


    return value::Array{Float64,2}, grad::Array{Float64,2}
end

log_studentt (generic function with 1 method)

In [6]:
# Evaluate stereo log prior.
# Set: alpha=1.0, sigma=1.0
function stereo_log_prior(x::Array{Float64,2})
    value = 0
    for i = 1:size(x)[1]
        for j = 1:size(x)[2]
            value

    return  value::Float64, grad::Array{Float64,2}
end


stereo_log_prior (generic function with 1 method)

In [7]:
# Evaluate stereo log likelihood.
# Set: Alpha = 1.0, Sigma = 0.004
function stereo_log_likelihood(x::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})


    return value::Float64, grad::Array{Float64,2}
end

stereo_log_likelihood (generic function with 1 method)

In [8]:
# Evaluate stereo posterior
function stereo_log_posterior(x::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})




    return log_posterior::Float64, log_posterior_grad::Array{Float64,2}
end

stereo_log_posterior (generic function with 1 method)

In [9]:
# Run stereo algorithm using gradient ascent or sth similar
function stereo(x0::Array{Float64,2}, im0::Array{Float64,2}, im1::Array{Float64,2})




    return x::Array{Float64,2}
end

stereo (generic function with 1 method)

In [10]:
function problem3()
    # use problem 2's load_data
    im0, im1, gt = load_data()

    # Display stereo: Initialized with constant 8's


    # Display stereo: Initialized with noise in [0,14]


    # Display stereo: Initialized with gt


    # Coarse to fine estimation..


end


problem3 (generic function with 1 method)

In [11]:
problem3()

([0.00622314 0.0108271 … 0.0108271 0.00622314; 0.0085251 0.0170502 … 0.0170502 0.0085251; … ; 0.133876 0.0584126 … 0.342575 0.168315; 0.0752275 0.0262337 … 0.183553 0.087269], [0.00622314 0.0108271 … 0.0108271 0.00622314; 0.0085251 0.0170502 … 0.015878 0.0085251; … ; 0.123007 0.0552165 … 0.334286 0.16092; 0.0685573 0.0239318 … 0.15898 0.0798314], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])